# 데이터셋 및 라이브러리 불러오기

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hypermodel import HyperModel

In [ ]:
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

import random

In [ ]:
import os
import shutil

base_path = "./../../data/processed/02_kaggle_dataset"
new_base = os.path.join(base_path, "train")

os.makedirs(os.path.join(new_base, "closed"), exist_ok=True)
os.makedirs(os.path.join(new_base, "open"), exist_ok=True)

# 기존 폴더에서 새 폴더로 복사
# shutil.move(os.path.join(base_path, "closed_eye"), os.path.join(new_base, "closed"))
# shutil.move(os.path.join(base_path, "open_eye"), os.path.join(new_base, "open"))

# 모델 구축

In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE=86
BATCH_SIZE=16
SEED=42

datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.7, 1.3],
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator=datagen.flow_from_directory(
    directory=new_base,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",  # 흑백 이미지
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
    shuffle=True,
    seed=SEED
)

val_generator=datagen.flow_from_directory(
    directory=new_base,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# from tensorflow.keras.optimizers import Adam

# model=Sequential([
#     Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
#     MaxPooling2D(2,2),
#     Conv2D(64, (3,3), activation='relu'),
#     MaxPooling2D(2,2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid')
# ])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model = Sequential()

# Conv Block 1
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(MaxPooling2D(2, 2))

# Conv Block 2
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Conv Block 3
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# FC Layer
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))  # 이진 분류

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# EarlyStopping: val_loss가 patience만큼 개선되지 않으면 멈춤
esc=EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)

# ModelCheckpoint: val_loss가 가장 낮은 시점마다 모델 저장
model_checkpoint=ModelCheckpoint("./../../src/models/02_Basic_Model/drowsiness_model.keras",
                                 monitor="val_loss", save_best_only=True, save_weights_only=False,
                                 mode='min', verbose=1)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    verbose=1,
    callbacks=[esc, model_checkpoint]
)

In [ ]:
# import numpy as np
# from sklearn.metrics import classification_report

# 예측값 추출
y_true = val_generator.classes

# 예측 수행 (steps 지정하면 더 안정적)
steps = val_generator.samples // val_generator.batch_size + 1
y_pred_prob = model.predict(val_generator, steps=steps)
y_pred = (y_pred_prob > 0.5).astype(int)

# 리포트 출력
print(classification_report(y_true, y_pred))

In [ ]:
# train_generator.class_indices

# 시각화

In [ ]:
# import matplotlib.pyplot as plt

# 정확도
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Accuracy Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# 손실
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Loss Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
imgs, labels = next(train_generator)

import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(imgs[i].squeeze(), cmap='gray')
    plt.title(f"Label: {int(labels[i])}")
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
model.save("./../../src/models/02_Basic_Model/drowsiness_model.keras")

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# import random

# 1. 검증 generator (shuffle=False)
vis_generator = datagen.flow_from_directory(
    directory=new_base,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

# 2. 전체 예측값
Y_pred = model.predict(vis_generator, verbose=0)
y_pred = (Y_pred > 0.5).astype(int).flatten()
y_true = vis_generator.classes
class_names = list(vis_generator.class_indices.keys())

# 3. 무작위로 인덱스 10개 추출
total_samples = len(y_true)
random_indices = random.sample(range(total_samples), 10)

# 4. 이미지 접근 및 시각화
plt.figure(figsize=(15, 6))
for i, idx in enumerate(random_indices):
    batch_index = idx // BATCH_SIZE
    offset = idx % BATCH_SIZE

    vis_generator.reset()
    for _ in range(batch_index + 1):
        images, _ = next(vis_generator)

    img = images[offset].squeeze()
    true_label = class_names[int(y_true[idx])]
    pred_label = class_names[int(y_pred[idx])]

    ax = plt.subplot(2, 5, i + 1)
    plt.imshow(img, cmap='gray')
    plt.title(f"실제: {true_label}\n예측: {pred_label}", fontsize=10)
    plt.axis("off")

plt.tight_layout()
plt.show()

# Keras Tuner

In [ ]:
# import os
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from kerastuner.tuners import RandomSearch
# from kerastuner.engine.hypermodel import HyperModel

# 이미지 설정
IMG_SIZE = 86
BATCH_SIZE = 16
SEED = 42
new_base = './../../data/processed/02_kaggle_dataset/train'

# 데이터 증강 설정
datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.7, 1.3],
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=new_base,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
    shuffle=True,
    seed=SEED
)

val_generator = datagen.flow_from_directory(
    directory=new_base,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

# HyperModel 클래스 정의
class CNNHyperModel(HyperModel):
    def build(self, hp):
        model = Sequential()

        # Conv Block 1
        model.add(Conv2D(hp.Int('conv_1_filter', 32, 64, step=16),
                         kernel_size=(3,3),
                         activation='relu',
                         input_shape=(IMG_SIZE, IMG_SIZE, 1)))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(2, 2))

        # Conv Block 2
        model.add(Conv2D(hp.Int('conv_2_filter', 64, 128, step=32),
                         kernel_size=(3,3),
                         activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(2, 2))

        # FC Layer
        model.add(Flatten())
        model.add(Dense(hp.Int('dense_units', 64, 256, step=64), activation='relu'))
        model.add(Dropout(hp.Float('dropout', 0.3, 0.6, step=0.1)))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(
            optimizer=Adam(hp.Float('lr', 1e-4, 1e-2, sampling='log')),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        return model

# Tuner 정의
tuner = RandomSearch(
    CNNHyperModel(),
    objective='val_accuracy',
    max_trials=10,  # 시도할 조합 수
    executions_per_trial=1,
    directory='tuner_results',
    project_name='eye_status_cnn'
)

# 콜백 설정
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

# 하이퍼파라미터 탐색 시작
tuner.search(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop, reduce_lr]
)

# 최적 모델 확인
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

In [ ]:
loss, acc = best_model.evaluate(val_generator)
print(f"Best Model Accuracy on Validation Set: {acc:.4f}")

In [ ]:
# 최적 trial 가져오기
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

# 각 하이퍼파라미터 값 출력
print("Best Hyperparameters:")
print(f"conv_1_filter: {best_hp.get('conv_1_filter')}")
print(f"conv_2_filter: {best_hp.get('conv_2_filter')}")
print(f"dense_units: {best_hp.get('dense_units')}")
print(f"dropout: {best_hp.get('dropout')}")
print(f"learning_rate: {best_hp.get('lr')}")

In [ ]:
tuner.results_summary()

In [ ]:
best_model.save("best_model.keras")